# Efficiency: locality; caching
_COSC 208, Introduction to Computer Systems, 2023-11-13_

## Announcements
* Project 3 due today @ 11pm

## Outline
* Warm-up
* Temporal vs. spatial locality
* CPU caches
* Cache replacement

## Warm-up

* Q1: _Cross-out unnecessary loads and stores in the following assembly code:_

```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20
    890:    str    w0, [sp, #12]    XXXXX
    894:    str    w1, [sp, #8]     XXXXX
    898:    ldr    w0, [sp, #12]    XXXXX
    89c:    ldr    w1, [sp, #8]     XXXXX
    8a0:    mul    w0, w1, w0
    8a4:    str    w0, [sp, #20]
    8a8:    mov    w0, #0x4b0
    8ac:    str    w0, [sp, #24]    XXXXX
    8b0:    ldr    w1, [sp, #20]
    8b4:    ldr    w0, [sp, #24]    XXXXX
    8b8:    sdiv   w0, w1, w0
    8bc:    str    w0, [sp, #28]    XXXXX
    8c0:    ldr    w0, [sp, #28]    XXXXX
    8c4:    add    sp, sp, #0x20
    8c8:    ret
```

```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20
    890:    str    w0, [sp, #12]
    894:    str    w1, [sp, #8]
    898:    ldr    w0, [sp, #12]
    89c:    ldr    w1, [sp, #8] 
    8a0:    mul    w0, w1, w0
    8a4:    str    w0, [sp, #20]
    8a8:    mov    w0, #0x4b0
    8ac:    str    w0, [sp, #24]
    8b0:    ldr    w1, [sp, #20]
    8b4:    ldr    w0, [sp, #24] 
    8b8:    sdiv   w0, w1, w0
    8bc:    str    w0, [sp, #28] 
    8c0:    ldr    w0, [sp, #28] 
    8c4:    add    sp, sp, #0x20
    8c8:    ret
```

🛑 **STOP here** after completing the above question; if you have extra time please **skip ahead** to the extra practice.

* Addtional load and store can be eliminated with better register assignments
```
000000000000088c <interest_due>:
    88c:    sub    sp, sp, #0x20    XXXXX
    8a0:    mul    w0, w1, w0       
    8a4:    str    w0, [sp, #20]    XXXXX
    8a8:    mov    w0, #0x4b0       // mov w1 #0x4b0
    8b0:    ldr    w1, [sp, #20]    XXXXX
    8b8:    sdiv   w0, w1, w0       // sdiv w0, w0, w1
    8c4:    add    sp, sp, #0x20    XXXXX
    8c8:    ret
```

## Temporal vs. spatial locality

* _What is temporal locality?_
    * Access the same data repeatedly
    * E.g., for loop variable
* _What is spatial locality?_
    * Access data with a similar scope
    * E.g., next item in array
    * E.g., local variables/parameters, which are stored in the same stack frame
* Analogies for temporal and spatial locality
    * Book storage (Dive Into Systems Section 11.3.2)
        * Temporal locality — store most frequently used books at your desk, less frequently used books on your bookshelf, and least frequently used books at the library
        * Spatial locality — checkout books on the same/nearby subjects when you go to the library
    * Groceries (pre-class questions 3 & 4)
        * Temporal locality — you store food you eat frequently in the front of the refrigerator, while you store food you eat infrequently in the back of the refrigerator
        * Spatial locality — you organize the items on your grocery list based on the aisle in which they are located
    * _With a partner, develop your own analogy for temporal and spatial locality_

## CPU caches

* _Why do we have caches on the CPU?_ — accessing main memory is ~100x slower than accessing a register
* Store instructions and data (stack, heap, etc.) from main memory
* Three levels — L1, L2, and L3
* Range in size from a few KB to a few MB – L1 is smallest and fastest; L3 is largest and slowest
* Cache line (i.e., cache entry) is typically larger than a word — e.g., 128 bytes
    * _Why?_ — spatial locality
* What happens when we write to memory?
    * Write through cache — write to the cache and main memory
    * Write back cache — initially write to the cache; write to main memory when the entry is evicted from the cache
    * What are the advantages of each approach?
        * Write through cache ensures consistency between CPU cores
        * Write back cache only incurs the overhead of accessing main memory when absolutely necessary

## Cache replacement

* If a cache is full, then a cache entry must be removed so different data can be placed in the cache
* Cache replacement policy governs which data is removed
* _What should a good cache replacement policy do?_ — maximize the number of cache hits (or minimize the number of cache misses)
    * Evaluation metric: Hit ratio = number of hits / total number of memory accesses
* _How do we determine which cache entry to replace?_
* Optimal replacement policy – replace the entry that will be accessed furthest in the future
    * Impractical because we don’t know data access patterns a priori
* Least Recently Used (LRU)
    * LRU assumes a item that was accessed recently will be accessed again soon – temporal locality
    * Downside: lots of overhead to implement — need to store an ordered list of items and move an item up in the list whenever it’s accessed
    * Where does this go wrong? — when working-set size (i.e., number of repeatedly accessed entries) is (slightly) greater than size of the cache
* First-in First-out (FIFO)
    * Simple to implement
    * Doesn’t consider the importance of a cache entry
* Random
    * Even simpler to implement
    * Doesn’t consider the importance of a cache entry

* Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
* Q2: _What is the sequence of hits, insertions, and replacements that occur when an **optimal** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -5/+2, H3, H4, -3/+1, H4, H4, H2, -1/+5, H2, H4
Hit ratio = 9/15 = 60%
```

<p style="height:6em;"></p>

* Q3: _What is the sequence of hits, insertions, and replacements that occur when a **first in first out (FIFO)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -3/+2, -4/+3, -5/+4, -2/+1, H4, H4, -3/+2, -4/+5, H2, -1/+4
Hit ratio = 5/15 = 33%
```

<p style="height:6em;"></p>

* Q4: _What is the sequence of hits, insertions, and replacements that occur when a **least recently used (LRU)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -4/+2, H3, -5/+4, -2/+1, H4, H4, -3/+2, -1/+5, H2, H4
Hit ratio = 7/15 = 47%
```

<p style="height:6em;"></p>

## Extra practice

* Q5: _Cross-out unnecessary loads and stores in the following assembly code:_

```
000000000040056c <volume>:                             
    40056c:    d10083ff     sub    sp, sp, #0x20       
    400570:    f9000bfe     str    x30, [sp, #16]      
    400574:    b9000fe0     str    w0, [sp, #12]       XXX
    400578:    b9000be1     str    w1, [sp, #8]        XXX
    40057c:    b90007e2     str    w2, [sp, #4]        
    400580:    b9400fe0     ldr    w0, [sp, #12]       XXX
    400584:    b9400be1     ldr    w1, [sp, #8]        XXX
    400588:    97ffffef     bl     400544 <multiply>   
    40058c:    b90003e0     str    w0, [sp]            XXX
    400590:    b94003e0     ldr    w0, [sp]            XXX
    400594:    b94007e1     ldr    w1, [sp, #4]        
    400598:    97ffffeb     bl     400544 <multiply>    
    40059c:    b90003e0     str    w0, [sp]            XXX
    4005a0:    b94003e0     ldr    w0, [sp]            XXX
    4005a4:    f9400bfe     ldr    x30, [sp, #16]      
    4005a8:    910083ff     add    sp, sp, #0x20       
    4005ac:    d65f03c0     ret                        
```

```
000000000040056c <volume>:                             
    40056c:    d10083ff     sub    sp, sp, #0x20       
    400570:    f9000bfe     str    x30, [sp, #16]      
    400574:    b9000fe0     str    w0, [sp, #12]       
    400578:    b9000be1     str    w1, [sp, #8]        
    40057c:    b90007e2     str    w2, [sp, #4]        
    400580:    b9400fe0     ldr    w0, [sp, #12]       
    400584:    b9400be1     ldr    w1, [sp, #8]        
    400588:    97ffffef     bl     400544 <multiply>    
    40058c:    b90003e0     str    w0, [sp]            
    400590:    b94003e0     ldr    w0, [sp]            
    400594:    b94007e1     ldr    w1, [sp, #4]        
    400598:    97ffffeb     bl     400544 <multiply>    
    40059c:    b90003e0     str    w0, [sp]            
    4005a0:    b94003e0     ldr    w0, [sp]            
    4005a4:    f9400bfe     ldr    x30, [sp, #16]      
    4005a8:    910083ff     add    sp, sp, #0x20       
    4005ac:    d65f03c0     ret                        
```

<div style="page-break-after:always;"></div>